In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, Reshape, Flatten, Conv2D, Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from PIL import Image
import os
from tensorflow.keras.models import load_model
from keras.layers import Dropout

In [2]:
IMG_SIZE = 64
LATENT_DIM = 100
BATCH_SIZE = 64  # Mantieni il batch size come era
EPOCHS = 10000
LR_D = 0.0002  # Learning rate per il discriminatore
LR_G = 0.0002  # Learning rate per il generatore

In [3]:
# Creazione del generatore
def build_generator():
    model = Sequential()
    model.add(Dense(256 * 16 * 16, activation="relu", input_dim=LATENT_DIM))
    model.add(Reshape((16, 16, 256)))  # Aumentata la capacità del generatore
    model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding="same", activation="relu"))
    model.add(Conv2DTranspose(64, kernel_size=3, strides=2, padding="same", activation="relu"))
    model.add(Conv2D(3, kernel_size=3, padding="same", activation="sigmoid"))  # Modifica a sigmoid
    model.compile(loss="binary_crossentropy", optimizer=Adam(LR_G))
    return model

# Creazione del discriminatore
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same", input_shape=(IMG_SIZE, IMG_SIZE, 3)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))  # Aggiunta di dropout per regolarizzazione
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss="binary_crossentropy", optimizer=Adam(LR_D), metrics=['accuracy'])
    return model

In [4]:
# Creazione del GAN
def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    model.compile(loss="binary_crossentropy", optimizer=Adam(LR_G))
    return model

def smooth_labels(y):
    return y - 0.1 + (np.random.random(y.shape) * 0.2)  # Etichette smooth tra 0.9 e 1.1


# Caricamento delle immagini
def load_images(path, img_size):
    images = []
    for filename in os.listdir(path):
        try:
            img = Image.open(os.path.join(path, filename)).resize((img_size, img_size))
            img = np.array(img)
            if img.shape == (img_size, img_size, 3):  # Controllo dimensioni corrette
                images.append(img)
        except Exception as e:
            print(f"Impossibile aprire il file {filename}: {e}")
    images = np.array(images)
    images = (images - 127.5) / 127.5  # Normalizzazione tra -1 e 1
    return images

# Salvataggio delle immagini generate
def save_generated_images(epoch, generator, latent_dim):
    noise = np.random.normal(0, 1, (25, latent_dim))
    generated_images = generator.predict(noise)
    generated_images = (generated_images + 1) / 2.0  # Rescale to [0, 1]

    for i in range(generated_images.shape[0]):
        img = (generated_images[i] * 255).astype(np.uint8)
        img = Image.fromarray(img)
        img.save(f"generated_cat_epoch_{epoch}_{i}.png")  # Salvare più immagini

def save_models(generator, discriminator, gan, epoch):
    save_dir = 'saved_models'
    os.makedirs(save_dir, exist_ok=True)
    
    generator.save(os.path.join(save_dir, f'generator_epoch_{epoch}.h5'))
    discriminator.save(os.path.join(save_dir, f'discriminator_epoch_{epoch}.h5'))
    gan.save(os.path.join(save_dir, f'gan_epoch_{epoch}.h5'))
    
    print(f"Models saved at epoch {epoch}")

In [5]:
# Allenamento della GAN
def train_gan(gan, generator, discriminator, dataset, latent_dim, epochs, batch_size):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Aggiorna il discriminatore
        for _ in range(1):  # Aggiorna il discriminatore una volta per ogni aggiornamento del generatore
            idx = np.random.randint(0, dataset.shape[0], half_batch)
            real_images = dataset[idx]
            real_labels = smooth_labels(np.ones((half_batch, 1)))  # Etichette smooth
            noise = np.random.normal(0, 1, (half_batch, latent_dim))
            fake_images = generator.predict(noise)
            fake_labels = np.zeros((half_batch, 1))

            d_loss_real = discriminator.train_on_batch(real_images, real_labels)
            d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
            d_loss = 0.5 * np.add(d_loss_real[0], d_loss_fake[0])

        # Aggiorna il generatore
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_y = np.ones((batch_size, 1))  # Usare etichette reali, non smooth
        g_loss = gan.train_on_batch(noise, valid_y)

        if epoch % 50 == 0:
            print(f"{epoch} [D loss: {d_loss}] [G loss: {g_loss}]")
            save_generated_images(epoch, generator, latent_dim)
        
        if epoch % 100 == 0:  # Ogni 50 epoche salva il modello attuale
            save_models(generator, discriminator, gan, epoch)

def load_saved_models(epoch):
    save_dir = 'saved_models'
    
    generator = load_model(os.path.join(save_dir, f'generator_epoch_{epoch}.h5'))
    discriminator = load_model(os.path.join(save_dir, f'discriminator_epoch_{epoch}.h5'))
    gan = load_model(os.path.join(save_dir, f'gan_epoch_{epoch}.h5'))
    
    print(f"Models loaded from epoch {epoch}")
    return generator, discriminator, gan

def generate_images(generator, latent_dim, num_images=5):
    noise = np.random.normal(0, 1, (num_images, latent_dim))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # Riscalare tra 0 e 1
    for i in range(num_images):
        img = Image.fromarray((generated_images[i] * 255).astype(np.uint8))
        img.save(f"generated_cat_{i}.png")

In [6]:
# NUOVO MODELLO
# generator = build_generator()
# discriminator = build_discriminator()
# gan = build_gan(generator, discriminator)
# dataset = load_images('C:/Users/Simone/OneDrive/Desktop/GAN_IMG_GEN_CATS/cats', IMG_SIZE)
# train_gan(gan, generator, discriminator, dataset, LATENT_DIM, EPOCHS, BATCH_SIZE)

In [7]:
# CARICARE UN MODELLO - GENERARE IMMAGINI
# epoch_to_load = 6600  # Sostituisci con l'epoca da cui vuoi caricare il modello
# generator, _, _ = load_saved_models(epoch_to_load)
# generate_images(generator, LATENT_DIM, num_images=1)

In [8]:
# CARICARE UN MODELLO - CONTINUARE L'ADDESTRAMENTO
epoch_to_load = 6600  # Sostituisci con l'epoca da cui vuoi caricare il modello
generator, discriminator, gan = load_saved_models(epoch_to_load)
dataset = load_images('C:/Users/Simone/OneDrive/Desktop/GAN_IMG_GEN_CATS/cats', IMG_SIZE)
remaining_epochs = EPOCHS - epoch_to_load
train_gan(gan, generator, discriminator, dataset, LATENT_DIM, remaining_epochs, BATCH_SIZE)


Models loaded from epoch 6600
1/1 [==============================] - 1s 1s/step

0 [D loss: -0.43787793861702085] [G loss: 5.170252799987793]
1/1 [==============================] - 0s 389ms/step


c:\Users\Simone\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Models saved at epoch 0
1/1 [==============================] - 0s 175ms/step
50 [D loss: 0.0036995445843786] [G loss: 3.35837984085083]
1/1 [==============================] - 0s 260ms/step
100 [D loss: 0.9929609398823231] [G loss: 1.96140718460083]
1/1 [==============================] - 0s 182ms/step
Models saved at epoch 100
1/1 [==============================] - 0s 178ms/step
150 [D loss: 0.14029858144931495] [G loss: 1.0250481367111206]
1/1 [==============================] - 0s 206ms/step
200 [D loss: -1.4767935993149877] [G loss: 0.37533044815063477]
1/1 [==============================] - 0s 173ms/step
Models saved at epoch 200
1/1 [==============================] - 0s 174ms/step
250 [D loss: 0.20042755990289152] [G loss: 0.2499360740184784]
1/1 [==============================] - 0s 167ms/step
300 [D loss: 0.1502990280278027] [G loss: 0.13625966012477875]
1/1 [==============================] - 0s 140ms/step
Models saved at epoch 300
1/1 [==============================] - 0s 167ms/s

KeyboardInterrupt: 